here, i i will attempt to find the bug that is preventing certain protein mutations from running quickly...

In [1]:
import simtk.openmm as openmm
from openmmtools.constants import kB
import simtk.unit as unit
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from perses.rjmc.topology_proposal import TopologyProposal, NetworkXMolecule
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from openmmtools import mcmc, utils
import openmmtools.cache as cache
from perses.dispersed.utils import configure_platform
#cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from perses.annihilation.lambda_protocol import LambdaProtocol
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
import openmmtools.integrators as integrators
from protein_test_cycle import *

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


and we should be using CUDA...

In [2]:
print(cache.global_context_cache.platform.getName())
print(cache.global_context_cache.platform.getPropertyDefaultValue('CudaPrecision'))

CUDA
mixed


In [ ]:
os.system(f"rm fast*.nc")
os.system(f"rm slow*.nc")


In [ ]:
def create_hss(reporter_name, hybrid_factory, selection_string ='all', checkpoint_interval = 1, n_states = 13):
    lambda_protocol = LambdaProtocol(functions='default')
    reporter = MultiStateReporter(reporter_name, analysis_particle_indices = hybrid_factory.hybrid_topology.select(selection_string), checkpoint_interval = checkpoint_interval)
    hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
                                                                                 collision_rate=5.0 / unit.picosecond,
                                                                                 n_steps=1,
                                                                                 reassign_velocities=False,
                                                                                 n_restart_attempts=20,
                                                                                 splitting="V R R R O R R R V",
                                                                                 constraint_tolerance=1e-06),
                                                                                 hybrid_factory=hybrid_factory, online_analysis_interval=10)
    hss.setup(n_states=n_states, temperature=300*unit.kelvin,storage_file=reporter,lambda_protocol=lambda_protocol,endstates=False)
    return hss, reporter

In [3]:
def deserialize(filename):
    import pickle
    with open(filename, 'rb') as f:
        htf = pickle.load(f)
    return htf



In [4]:
fast_htf = deserialize('ALA_SER.vac.pkl')
slow_htf = deserialize('ALA_THR.vac.pkl') 
# fast_hss, _ = create_hss(f"fast_debug.nc", fast_htf)
# slow_hss, _ = create_hss(f"slow_debug.nc", slow_htf)

In [ ]:
fast_hss.extend(2)
print(f"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
slow_hss.extend(2)

as you can see above, even in vacuum, there is a big difference in the replica propagation time between the fast and slow systems, even in vacuum;
perhaps if we remove the constraints in the slow system, we can fix the problem

In [ ]:
import copy

In [15]:
slow_htf_mod = copy.deepcopy(slow_htf)


In [16]:
slow_htf._atom_classes


{'unique_old_atoms': {11, 12, 13},
 'unique_new_atoms': {22, 23, 24, 25, 26, 27, 28},
 'core_atoms': {4, 6, 7, 8, 9, 10, 14, 15, 16},
 'environment_atoms': {0, 1, 2, 3, 5, 17, 18, 19, 20, 21}}

In [17]:
num_constraints = slow_htf_mod._hybrid_system.getNumConstraints()

In [18]:
num_constraints

17

In [ ]:
def create_new_pdb(topology, positions, output_pdb = 'test_new.pdb'):
    """
    create a pdb of the geometry proposal (only new system)
    """
    import mdtraj as md
    _positions =  np.array(positions.value_in_unit(unit.nanometer))
    print(_positions)
    traj = md.Trajectory(_positions, md.Topology.from_openmm(topology))
    traj.save(output_pdb)

In [19]:
slow_htf_mod._hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2acb936a5db0> >,
 <simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2acb936a56c0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2acb936a5d80> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2acb936a5f00> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2acb936a5ea0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2acb936a5ed0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2acb936a5c30> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2acb936a5cf0> >,
 <simtk.open

In [ ]:
num_bonds = slow_htf._hybrid_system.getForce(0).getNumBonds()
for i in range(num_bonds):
    print(slow_htf._hybrid_system.getForce(0).getBondParameters(i))

In [ ]:
num_bonds = slow_htf._hybrid_system.getForce(1).getNumBonds()
for i in range(num_bonds):
    print(slow_htf._hybrid_system.getForce(1).getBondParameters(i))

In [ ]:
slow_htf_mod._hybrid_topology
table, bonds = md.Topology.from_openmm(slow_htf_mod._topology_proposal._old_topology).to_dataframe()
print(table)

In [ ]:
bonds

In [ ]:
for i in range(num_constraints):
    params = slow_htf_mod._hybrid_system.getConstraintParameters(i)
    print(params)

In [ ]:
for i in reversed(range(num_constraints)):
    slow_htf_mod._hybrid_system.removeConstraint(i)
    

In [ ]:
slow_hss_mod, _ = create_hss(f"slow_mod_debug.nc", slow_htf_mod)

In [ ]:
slow_hss_mod.extend(5)

In [9]:
def create_integrator(htf, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
    fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))
    
    fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
    
    fast_sampler_state = SamplerState(positions = htf._hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
    
    integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
                                                     timestep = 4.0* unit.femtoseconds,
                                                     splitting = 'V R O R V',
                                                     measure_shadow_work = False,
                                                     measure_heat = False,
                                                     constraint_tolerance = constraint_tol,
                                                     collision_rate = 5.0 / unit.picoseconds)
    
    fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
    fast_sampler_state.apply_to_context(fast_context)
    
    return fast_context, fast_thermodynamic_state, fast_sampler_state, fast_integrator


    
    
    
    

In [10]:
_, _, _, fast_int = create_integrator(fast_htf, 1e-6)
_, _, _, slow_int = create_integrator(slow_htf, 1e-6)
_, _, _, slow_int_mod1 = create_integrator(copy.deepcopy(slow_htf), 1e-5)
_, _, _, slow_int_mod2 = create_integrator(copy.deepcopy(slow_htf), 1e-4)
_, _, _, slow_int_mod3 = create_integrator(copy.deepcopy(slow_htf), 1e-3)

In [11]:
def time_integrator(integrator, num_steps = 10):
    import time
    _time = []
    integrator.step(1)
    for i in range(num_steps):
        start = time.time()
        integrator.step(1)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

In [12]:
fast = time_integrator(fast_int, num_steps = 100)
slow = time_integrator(slow_int, num_steps = 100)
slow_mod1 = time_integrator(slow_int_mod1, num_steps = 100)
slow_mod2 = time_integrator(slow_int_mod2, num_steps = 100)
slow_mod3 = time_integrator(slow_int_mod3, num_steps = 100)

In [14]:
print(np.average(fast), np.std(fast))
print(np.average(slow), np.std(slow))
print(np.average(slow_mod1), np.std(slow_mod1))
print(np.average(slow_mod2), np.std(slow_mod2))
print(np.average(slow_mod3), np.std(slow_mod3))
#print(np.average(slow_mod), np.std(slow_mod))

0.00016542673110961915 8.413250400535065e-05
0.0010826468467712402 8.570893828375499e-05
0.001002357006072998 4.801813747161436e-05
0.0010527682304382324 0.00024120706113980742
0.0009982705116271973 5.470035517064183e-05


^^^^^^^ perhaps if we remove the constraints, it will run faster? and lo, it does




## now, we have to figure out how we can fix this issue; let's up the constraint tolerance iteratively...


i'll iterate over a few orders of magnitude with the constraint tolerance to try to pull up the speed of integration...

In [ ]:
for tol in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]:
    _, _, _, slow_int = create_integrator(fast_htf, tol)
    